In [1]:
import pandas as pd

# Carga de datasets
df_base_clientes = pd.read_csv('../../dataset/base_clientes.csv', sep=',')
df_base_consumos = pd.read_csv('../../dataset/base_consumos.csv', sep=',')
df_base_saldos_creditos = pd.read_csv('../../dataset/base_saldos_creditos.csv', sep=',')
df_base_saldos_cuentas = pd.read_csv('../../dataset/base_saldos_cuentas.csv', sep=',')
df_universo = pd.read_csv('../../dataset/universo.csv', sep=',')

# Verificación rápida
print("Datasets cargados correctamente:")
print(f"base_clientes → {df_base_clientes.shape}")
print(f"base_consumos → {df_base_consumos.shape}")
print(f"base_saldos_creditos → {df_base_saldos_creditos.shape}")
print(f"base_saldos_cuentas → {df_base_saldos_cuentas.shape}")
print(f"universo → {df_universo.shape}")


Datasets cargados correctamente:
base_clientes → (809579, 14)
base_consumos → (3697738, 7)
base_saldos_creditos → (11597785, 8)
base_saldos_cuentas → (8876560, 5)
universo → (535943, 3)


In [2]:

# Mostrar columnas de cada dataset
print("\nColumnas de cada DataFrame:")
print("\n🟦 df_base_clientes.columns:")
print(df_base_clientes.columns.tolist())

print("\n🟩 df_base_consumos.columns:")
print(df_base_consumos.columns.tolist())

print("\n🟨 df_base_saldos_creditos.columns:")
print(df_base_saldos_creditos.columns.tolist())

print("\n🟧 df_base_saldos_cuentas.columns:")
print(df_base_saldos_cuentas.columns.tolist())

print("\n🟪 df_universo.columns:")
print(df_universo.columns.tolist())


Columnas de cada DataFrame:

🟦 df_base_clientes.columns:
['ID', 'edad', 'ingreso', 'anios_exp', 'tipo_empleo', 'buro_de_credito', 'pago_nomina', 'tc', 'prest_per', 'prest_veh_hip', 'cuenta_ahorro', 'ofer_prest_per', 'ofer_tc', 'ofer_compra_deuda']

🟩 df_base_consumos.columns:
['periodo', 'ID', 'mes', 'compra_supermercado', 'compra_restaurante', 'compra_salud', 'compra_transporte']

🟨 df_base_saldos_creditos.columns:
['mes', 'ID', 'tipo_credito', 'tipo_producto', 'inst_fin', 'monto_saldo', 'dias_atraso', 'periodo']

🟧 df_base_saldos_cuentas.columns:
['mes', 'ID', 'saldo_ahorros', 'saldo_otros', 'periodo']

🟪 df_universo.columns:
['ID', 'target', 'periodo']


In [3]:
# Análisis de IDs: df_universo vs otros dataframes

# Obtener IDs únicos de cada dataframe
ids_universo = set(df_universo['ID'].unique())
ids_clientes = set(df_base_clientes['ID'].unique())
ids_consumos = set(df_base_consumos['ID'].unique())
ids_creditos = set(df_base_saldos_creditos['ID'].unique())
ids_cuentas = set(df_base_saldos_cuentas['ID'].unique())

print("=" * 80)
print("📊 ANÁLISIS DE COBERTURA DE IDs")
print("=" * 80)
print(f"\n🎯 Total de IDs únicos en df_universo: {len(ids_universo):,}")
print("\n" + "-" * 80)

# Contar IDs de df_universo que NO están en cada dataframe
missing_in_clientes = ids_universo - ids_clientes
missing_in_consumos = ids_universo - ids_consumos
missing_in_creditos = ids_universo - ids_creditos
missing_in_cuentas = ids_universo - ids_cuentas

print("\n🔍 IDs del UNIVERSO que NO están en cada tabla:")
print(f"   • base_clientes:        {len(missing_in_clientes):,} IDs faltantes")
print(f"   • base_consumos:        {len(missing_in_consumos):,} IDs faltantes")
print(f"   • base_saldos_creditos: {len(missing_in_creditos):,} IDs faltantes")
print(f"   • base_saldos_cuentas:  {len(missing_in_cuentas):,} IDs faltantes")
print("\n" + "-" * 80)

# Calcular cobertura
print("\n✅ Cobertura de IDs (% de universo presente en cada tabla):")
print(f"   • base_clientes:        {(1 - len(missing_in_clientes)/len(ids_universo))*100:.2f}%")
print(f"   • base_consumos:        {(1 - len(missing_in_consumos)/len(ids_universo))*100:.2f}%")
print(f"   • base_saldos_creditos: {(1 - len(missing_in_creditos)/len(ids_universo))*100:.2f}%")
print(f"   • base_saldos_cuentas:  {(1 - len(missing_in_cuentas)/len(ids_universo))*100:.2f}%")
print("\n" + "=" * 80)

📊 ANÁLISIS DE COBERTURA DE IDs

🎯 Total de IDs únicos en df_universo: 535,943

--------------------------------------------------------------------------------

🔍 IDs del UNIVERSO que NO están en cada tabla:
   • base_clientes:        0 IDs faltantes
   • base_consumos:        187,092 IDs faltantes
   • base_saldos_creditos: 203,288 IDs faltantes
   • base_saldos_cuentas:  12,297 IDs faltantes

--------------------------------------------------------------------------------

✅ Cobertura de IDs (% de universo presente en cada tabla):
   • base_clientes:        100.00%
   • base_consumos:        65.09%
   • base_saldos_creditos: 62.07%
   • base_saldos_cuentas:  97.71%



In [4]:
# Cuadro comparativo: LEFT JOIN vs INNER JOIN
import pandas as pd

# Crear tabla comparativa
comparacion_data = []

# Para cada dataframe, calcular cuántos registros quedarían
dataframes = {
    'base_clientes': df_base_clientes,
    'base_consumos': df_base_consumos,
    'base_saldos_creditos': df_base_saldos_creditos,
    'base_saldos_cuentas': df_base_saldos_cuentas
}

print("\n" + "=" * 100)
print("📋 CUADRO COMPARATIVO: LEFT JOIN vs INNER JOIN (universo con cada tabla)")
print("=" * 100)

for nombre, df_other in dataframes.items():
    # LEFT JOIN: mantiene todos los registros de df_universo
    registros_left = len(df_universo)
    
    # INNER JOIN: solo registros con ID en ambas tablas
    ids_comunes = ids_universo.intersection(set(df_other['ID'].unique()))
    registros_universo_inner = df_universo[df_universo['ID'].isin(ids_comunes)].shape[0]
    
    # Pérdida de registros
    perdida = registros_left - registros_universo_inner
    perdida_pct = (perdida / registros_left) * 100
    
    comparacion_data.append({
        'DataFrame': nombre,
        'LEFT JOIN (universo)': f"{registros_left:,}",
        'INNER JOIN (universo)': f"{registros_universo_inner:,}",
        'Registros Perdidos': f"{perdida:,}",
        'Pérdida %': f"{perdida_pct:.2f}%"
    })

# Crear y mostrar tabla
df_comparacion = pd.DataFrame(comparacion_data)
print("\n")
print(df_comparacion.to_string(index=False))
print("\n" + "=" * 100)
print(f"\n💡 CONCLUSIÓN:")
print(f"   Con LEFT JOIN mantienes los {len(df_universo):,} registros del universo (target completo)")
print(f"   Con INNER JOIN pierdes registros donde el ID no existe en la otra tabla")
print("=" * 100)


📋 CUADRO COMPARATIVO: LEFT JOIN vs INNER JOIN (universo con cada tabla)


           DataFrame LEFT JOIN (universo) INNER JOIN (universo) Registros Perdidos Pérdida %
       base_clientes              535,943               535,943                  0     0.00%
       base_consumos              535,943               348,851            187,092    34.91%
base_saldos_creditos              535,943               332,655            203,288    37.93%
 base_saldos_cuentas              535,943               523,646             12,297     2.29%


💡 CONCLUSIÓN:
   Con LEFT JOIN mantienes los 535,943 registros del universo (target completo)
   Con INNER JOIN pierdes registros donde el ID no existe en la otra tabla
